In [1]:
import pandas as pd
from yaml import load
from yaml import CLoader as Loader
from sklearn.preprocessing import OneHotEncoder

In [2]:
#If you want to debug the package without repackaging, you can read files locally:
import sys
sys.path.insert(0, '/home/phyto/Abil/abil/')
from tune import tune
from functions import example_data, upsample

In [3]:
# Setting up a regressor model
with open('/home/phyto/Abil/configuration/ensemble_2-phase.yml', 'r') as f:
    model_config = load(f, Loader=Loader)

predictors = model_config['predictors']
#read your target + environmental data:
d = pd.read_csv("/home/phyto/Abil/devries2024/data/obs_env.csv")
species =  "Emiliania huxleyi"
d[species] = d[species].fillna(0)
d = d.dropna(subset=[species])
d = d.dropna(subset=['FID'])
d = upsample(d, species, ratio=10)
print(len(d[d[species]==0]))
print(len(d[d[species]>0]))

X_train = d[predictors]
y = d[species]

m = tune(X_train, y, model_config, regions="FID")

13940
1394


In [4]:
'''
1-phase XGboost regressor
'''
m.train(model="xgb", regressor=True, classifier=True)

training classifier
[0. 0. 0. ... 0. 1. 0.]
Fitting 10 folds for each of 1 candidates, totalling 10 fits


In [ ]:
'''
1-phase Random forest regressor
'''
m.train(model="rf", regressor=True)

training regressor
{'R2': 'r2', 'MAE': 'neg_mean_absolute_error', 'RMSE': 'neg_root_mean_squared_error', 'tau': make_scorer(tau_scoring), 'tau_p': make_scorer(tau_scoring_p)}
{'regressor__estimator__n_estimators': [100], 'regressor__estimator__max_features': [4], 'regressor__estimator__max_depth': [50], 'regressor__estimator__min_samples_leaf': [0.5, 1], 'regressor__estimator__max_samples': [0.5, 1]}
Fitting 10 folds for each of 4 candidates, totalling 40 fits


exported model to: /home/phyto/Abil/ModelOutput/rf/model/Emiliania huxleyi_reg.sav
exported scoring to: /home/phyto/Abil/ModelOutput/rf/scoring/Emiliania huxleyi_reg.sav
reg rRMSE: 236%
reg rMAE: 64%
reg R2: 0.61
reg tau: 0.6
execution time: 7.152557373046875e-07 seconds


In [ ]:
'''
1-phase KNN regressor
'''
m.train(model="knn", regressor=True)

training regressor
{'R2': 'r2', 'MAE': 'neg_mean_absolute_error', 'RMSE': 'neg_root_mean_squared_error', 'tau': make_scorer(tau_scoring), 'tau_p': make_scorer(tau_scoring_p)}
{'regressor__estimator__max_samples': [0.8], 'regressor__estimator__max_features': [0.8], 'regressor__estimator__estimator__leaf_size': [5], 'regressor__estimator__estimator__n_neighbors': [5], 'regressor__estimator__estimator__p': [1], 'regressor__estimator__estimator__weights': ['uniform', 'distance']}
Fitting 10 folds for each of 2 candidates, totalling 20 fits


KeyboardInterrupt: 